In [2]:
import logging
import os
import argparse
import math
import random
import tqdm
import numpy as np
import pandas as pd
from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
import matplotlib.pyplot as plt
import torch.nn as nn

from script import dataloader, utility, earlystopping
from model import models, layers


In [ ]:
from main import *

In [ ]:
class dilated_inception(nn.Module):
    def __init__(self, cin, cout, dilation_factor=2):
        super(dilated_inception, self).__init__()
        self.tconv = nn.ModuleList()
        self.kernel_set = [2,3,6,7]
        cout = int(cout/len(self.kernel_set))
        for kern in self.kernel_set:
            self.tconv.append(nn.Conv2d(cin,cout,(1,kern),dilation=(1,dilation_factor)))

    def forward(self,input):
        x = []
        for i in range(len(self.kernel_set)):
            x.append(self.tconv[i](input))
        for i in range(len(self.kernel_set)):
            x[i] = x[i][...,-x[-1].size(3):]
        x = torch.cat(x,dim=1)
        return x

In [ ]:
logging.basicConfig(level=logging.INFO)

args, device, blocks = get_parameters()
n_vertex, zscore, train_iter, val_iter, test_iter = data_preparate(args, device)    # n_vertex = 207,

for x, y in tqdm.tqdm(train_iter):
    num = 0

print(x.shape)

In [26]:
kern = 2
dilation_factor = 1
cin = 12
cout = 16

TCconv = nn.ModuleList()
kernel_set = [2,3,4,5]

x = torch.rand(32,1,12,207)
cin = x.shape[1]

conv1 = nn.Conv2d(cin,cout,kernel_size=(1,2))
conv2 = nn.Conv2d(cin,cout,kernel_size=(1,3))
conv3 = nn.Conv2d(cin,cout,kernel_size=(1,4))
conv4 = nn.Conv2d(cin,cout,kernel_size=(1,5))

In [20]:
x.permute(0,1,3,2).shape

torch.Size([32, 1, 207, 12])

In [27]:
print(conv1(x.permute(0,1,3,2)).shape)
print(conv2(x.permute(0,1,3,2)).shape)
print(conv3(x.permute(0,1,3,2)).shape)
print(conv4(x.permute(0,1,3,2)).shape)

torch.Size([32, 16, 207, 11])
torch.Size([32, 16, 207, 10])
torch.Size([32, 16, 207, 9])
torch.Size([32, 16, 207, 8])


In [28]:
x1 = conv1(x.permute(0,1,3,2))
x2 = conv2(x.permute(0,1,3,2))
x3 = conv3(x.permute(0,1,3,2))
x4 = conv4(x.permute(0,1,3,2))

In [35]:
out = []
out.append(x1[...,-x4.shape[-1]:])
out.append(x2[...,-x4.shape[-1]:])
out.append(x3[...,-x4.shape[-1]:])
out.append(x4[...,-x4.shape[-1]:])

In [36]:
torch.cat(out,dim=1).shape

torch.Size([32, 64, 207, 8])

In [ ]:
for kernel_size in kernel_set:
    TCconv.append(nn.Conv2d(cin,cout,kernel_size=(1,kernel_size),dilation=(1,dilation_factor)))

In [38]:
x.permute(0,1,3,2).shape

torch.Size([32, 1, 207, 12])

In [40]:
class Dialated_Block(nn.Module):
    def __init__(self, cin, cout, dilation_factor=1):
        super(Dialated_Block,self).__init__()
        self.TCconv = nn.ModuleList()
        self.kernel_set = [2,3,4,5]
        cout = int(cout / len(self.kernel_set))
        for kernel_size in self.kernel_set:
            self.TCconv.append(nn.Conv2d(cin,cout,kernel_size=(1,kernel_size),dilation=(1,dilation_factor)))
        
    def forward(self,input):
        x = []
        for i in range(len(self.kernel_set)):
            x.append(self.TCconv[i](input))
            
        for i in range(len(self.kernel_set)):
            x[i] = x[i][...,-x[-1].shape[-1]:]
        x = torch.cat(x,dim=1)
        x = x.permute(0,1,3,2)
        return x


In [45]:
class Dialated_Block(nn.Module):
    def __init__(self, cin, cout, dilation_factor=1):
        super(Dialated_Block,self).__init__()
        self.TCconv = nn.ModuleList()
        self.kernel_set = [2,3]
        cout = int(cout / len(self.kernel_set))
        for kernel_size in self.kernel_set:
            self.TCconv.append(nn.Conv2d(cin,cout,kernel_size=(1,kernel_size),dilation=(1,dilation_factor)))
        
    def forward(self,input):
        input = input.permute(0,1,3,2)
        x = []
        
        for i in range(len(self.kernel_set)):
            x.append(self.TCconv[i](input))
        for i in range(len(self.kernel_set)):
            x[i] = x[i][...,-x[-1].shape[-1]:]
        x = torch.cat(x,dim=1)
        x = x.permute(0,1,3,2)
        return x


In [46]:
x.shape

torch.Size([32, 1, 12, 207])

In [41]:
tep_layer = Dialated_Block(1,64)

In [42]:
tep_layer

Dialated_Block(
  (TCconv): ModuleList(
    (0): Conv2d(1, 16, kernel_size=(1, 2), stride=(1, 1))
    (1): Conv2d(1, 16, kernel_size=(1, 3), stride=(1, 1))
    (2): Conv2d(1, 16, kernel_size=(1, 4), stride=(1, 1))
    (3): Conv2d(1, 16, kernel_size=(1, 5), stride=(1, 1))
  )
)

In [44]:
tep_layer(x.permute(0,1,3,2)).shape  

torch.Size([32, 64, 207, 8])

In [47]:
class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()

In [52]:
embeds = FixedEmbedding(1,128)

In [57]:
c_in = 1
d_model = 128

w = torch.zeros(c_in, d_model).float()
position = torch.arange(0, c_in).float().unsqueeze(1)
div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

In [58]:
div_term.shape

torch.Size([64])

In [64]:
w.shape

torch.Size([1, 128])

In [67]:
nn.Embedding(c_in, d_model)

Embedding(1, 128)

In [68]:
embeds.emb.weight = nn.Parameter(w, requires_grad=False)

In [70]:
embeds = nn.Embedding(1,128)

In [77]:
embeds(x.to(torch.int64)).shape

torch.Size([32, 1, 12, 207, 128])

In [79]:
z = torch.rand(32,1)

In [85]:
embeds(x.to(torch.int64)).squeeze().permute(0,3,1,2).shape

torch.Size([32, 128, 12, 207])

In [86]:
def plot_losses(train_losses, val_losses):

    
    epochs = list(range(1, len(train_losses) + 1))

    fig, ax = plt.subplots(figsize=(16, 8)) 

    plt.plot(epochs, train_losses, marker='o', linestyle='-', colo11r='green', label='Train')
    plt.plot(epochs, val_losses, marker='o', linestyle='-', color='red', label='Validation')

    plt.xticks(np.arange(1, len(epochs) + 1, step=1), rotation=90)

    plt.title('Loss(Mean Squared Error)')

    plt.xlabel('Epoch')

    plt.legend(loc='upper right')

    plt.savefig('STGCN Learning Curves-Default.png')

In [87]:
train_losses = [0.3417280171318853, 0.26110783354576567] 

val_losses = [0.35265180468559265, 0.40453770756721497]

In [100]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]

In [93]:
pe = torch.zeros(64, d_model).float()
position = torch.arange(0, 64).float().unsqueeze(1)

In [94]:
div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

In [95]:
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)

In [101]:
pe = PositionalEmbedding(d_model=d_model,max_len=32)

In [106]:
pe(x)[0][0]

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1.])

In [117]:
class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, time_of_day_size, day_of_week_size, day_of_month_size, day_of_year_size, embed_type='fixed'):
        super(TemporalEmbedding, self).__init__()

        Embed = FixedEmbedding if embed_type=='fixed' else nn.Embedding
        self.time_of_day_size = time_of_day_size
        self.day_of_week_size = day_of_week_size
        self.day_of_month_size = day_of_month_size
        self.day_of_year_size = day_of_year_size

        if time_of_day_size is not None:
            self.time_of_day_embed = Embed(time_of_day_size, d_model)
        if day_of_week_size is not None:
            self.day_of_week_embed = Embed(day_of_week_size, d_model)
        if day_of_month_size is not None:
            self.day_of_month_embed = Embed(day_of_month_size, d_model)
        if day_of_year_size is not None:
            self.day_of_year_embed = Embed(day_of_year_size, d_model)

    def forward(self, x):
        x = x + 0.5 # re-normalize to [0, 1) to get correct index. ref: arch_zoo/utils/xformer.py data_transformation_4_xformer
        temporal_embeddings = []
        if self.time_of_day_size is not None:
            time_of_day_x = self.time_of_day_embed((x[:, :, 0] * self.time_of_day_size).long())
            temporal_embeddings.append(time_of_day_x)
        if self.day_of_week_size is not None:
            day_of_week_x = self.day_of_week_embed((x[:, :, 1] * self.day_of_week_size).long())
            temporal_embeddings.append(day_of_week_x)
        if self.day_of_month_size is not None:
            day_of_month_x = self.day_of_month_embed((x[:, :, 2] * self.day_of_month_size).long())
            temporal_embeddings.append(day_of_month_x)
        if self.day_of_year_size is not None:
            day_of_year_x = self.day_of_year_embed((x[:, :, 3] * self.day_of_year_size).long())
            temporal_embeddings.append(day_of_year_x)

        if len(temporal_embeddings) == 0:
            return 0
        else:
            return sum(temporal_embeddings)

In [122]:
x = torch.rand(32721)

In [124]:
dataset_path = './data'
dataset_name = "metr-la"
dataset_path = os.path.join(dataset_path, dataset_name)
vel = pd.read_csv(os.path.join(dataset_path, 'vel.csv'))



In [125]:
vel.shape

(34271, 207)

In [141]:
ts_index = vel.index 

In [142]:
time_of_hour_size = 12          # 12
time_of_day_size = 12 * 24      # 288
time_of_week_size = 12* 24 * 7  # 2016

In [143]:
ts_hour = ts_index % time_of_hour_size
ts_day = ts_index % time_of_day_size
ts_week = ts_index % time_of_week_size

ts_hour_sin = np.sin(2* np.pi * ts_hour / ts_hour.max())
ts_hour_cos = np.cos(2* np.pi * ts_hour / ts_hour.max())
ts_day_sin = np.sin(2* np.pi * ts_day / ts_day.max())
ts_day_cos = np.cos(2* np.pi * ts_day / ts_day.max())
ts_week_sin = np.sin(2* np.pi * ts_week / ts_week.max())
ts_week_cos = np.cos(2* np.pi * ts_week / ts_week.max())

In [144]:
temporal_embeds = np.vstack([ts_hour_sin, ts_hour_cos,ts_day_sin,ts_day_cos,ts_week_sin,ts_week_cos]).T
temporal_embeds_sum = temporal_embeds.sum(axis=1)
data_after_embeds = np.repeat([temporal_embeds_sum], vel.shape[1], axis=0).T + vel

In [ ]:
def embeds(data, time_of_hour=True, time_of_day = True, time_of_week = True):
    ts_index = data.index 

    if time_of_hour:
        time_of_hour_size = 12          # 12
        ts_hour = ts_index % time_of_hour_size
        ts_hour_sin = np.sin(2* np.pi * ts_hour / ts_hour.max())
        ts_hour_cos = np.cos(2* np.pi * ts_hour / ts_hour.max())

    if time_of_day:
        time_of_day_size = 12 * 24      # 288
        ts_day = ts_index % time_of_day_size
        ts_day_sin = np.sin(2* np.pi * ts_day / ts_day.max())
        ts_day_cos = np.cos(2* np.pi * ts_day / ts_day.max())

    if time_of_week:
        time_of_week_size = 12* 24 * 7  # 2016
        ts_week = ts_index % time_of_week_size
        ts_week_sin = np.sin(2* np.pi * ts_week / ts_week.max())
        ts_week_cos = np.cos(2* np.pi * ts_week / ts_week.max())


    if time_of_hour and time_of_day and time_of_week:
        temporal_embeds = np.vstack([ts_hour_sin, ts_hour_cos,ts_day_sin,ts_day_cos,ts_week_sin,ts_week_cos]).T

    elif time_of_hour and (not time_of_day) and (not time_of_week):
        temporal_embeds = np.vstack([ts_hour_sin, ts_hour_cos]).T
    
    elif time_of_day and (not time_of_hour) and (not time_of_week):
        temporal_embeds = np.vstack([ts_day_sin,ts_day_cos]).T

    elif time_of_week and (not time_of_hour) and (not time_of_day):
        temporal_embeds = np.vstack([ts_week_sin,ts_week_cos]).T

    temporal_embeds_sum = temporal_embeds.sum(axis=1)
    data_after_embeds = np.repeat([temporal_embeds_sum], vel.shape[1], axis=0).T + vel


    return data_after_embeds

In [177]:

def get_parameters():
    parser = argparse.ArgumentParser(description='STGCN')
    parser.add_argument('--enable_cuda', type=bool, default=True, help='enable CUDA, default as True')
    parser.add_argument('--seed', type=int, default=42, help='set the random seed for stabilizing experiment results')
    parser.add_argument('--dataset', type=str, default='metr-la', choices=['metr-la', 'pems-bay', 'pemsd7-m'])
    parser.add_argument('--n_his', type=int, default=12)
    parser.add_argument('--n_pred', type=int, default=3, help='the number of time interval for predcition, default as 3')
    parser.add_argument('--time_intvl', type=int, default=5)
    parser.add_argument('--Kt', type=int, default=2)
    parser.add_argument('--stblock_num', type=int, default=2)
    parser.add_argument('--act_func', type=str, default='glu', choices=['glu', 'gtu'])
    parser.add_argument('--Ks', type=int, default=3, choices=[3, 2])
    parser.add_argument('--graph_conv_type', type=str, default='cheb_graph_conv', choices=['cheb_graph_conv', 'graph_conv'])
    parser.add_argument('--gso_type', type=str, default='sym_norm_lap', choices=['sym_norm_lap', 'rw_norm_lap', 'sym_renorm_adj', 'rw_renorm_adj'])
    parser.add_argument('--enable_bias', type=bool, default=True, help='default as True')
    parser.add_argument('--droprate', type=float, default=0.5)
    parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
    parser.add_argument('--weight_decay_rate', type=float, default=0.0005, help='weight decay (L2 penalty)')
    parser.add_argument('--batch_size', type=int, default=32)
    parser.add_argument('--epochs', type=int, default=100, help='epochs, default as 100')
    parser.add_argument('--opt', type=str, default='adam', help='optimizer, default as adam')
    parser.add_argument('--step_size', type=int, default=10)
    parser.add_argument('--gamma', type=float, default=0.95)
    parser.add_argument('--patience', type=int, default=30, help='early stopping patience')
    args = parser.parse_args()
    print('Training configs: {}'.format(args))

    # For stable experiment results
    set_env(args.seed)

    # Running in Nvidia GPU (CUDA) or CPU
    if args.enable_cuda and torch.cuda.is_available():
        # Set available CUDA devices
        # This option is crucial for multiple GPUs
        # 'cuda' ≡ 'cuda:0'
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    
    Ko = args.n_his - (args.Kt - 1) * 2 * args.stblock_num

    # blocks: settings of channel size in st_conv_blocks and output layer,
    # using the bottleneck design in st_conv_blocks
    blocks = []
    blocks.append([1])
    for l in range(args.stblock_num):  # 2
        blocks.append([64, 16, 64])
    if Ko == 0:
        blocks.append([128])
    elif Ko > 0:
        blocks.append([128, 128])
    blocks.append([1])
    
    return args, device, blocks


In [178]:
args, device, blocks = get_parameters()

usage: ipykernel_launcher.py [-h] [--enable_cuda ENABLE_CUDA] [--seed SEED]
                             [--dataset {metr-la,pems-bay,pemsd7-m}]
                             [--n_his N_HIS] [--n_pred N_PRED]
                             [--time_intvl TIME_INTVL] [--Kt KT]
                             [--stblock_num STBLOCK_NUM]
                             [--act_func {glu,gtu}] [--Ks {3,2}]
                             [--graph_conv_type {cheb_graph_conv,graph_conv}]
                             [--gso_type {sym_norm_lap,rw_norm_lap,sym_renorm_adj,rw_renorm_adj}]
                             [--enable_bias ENABLE_BIAS] [--droprate DROPRATE]
                             [--lr LR] [--weight_decay_rate WEIGHT_DECAY_RATE]
                             [--batch_size BATCH_SIZE] [--epochs EPOCHS]
                             [--opt OPT] [--step_size STEP_SIZE]
                             [--gamma GAMMA] [--patience PATIENCE]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/iv

SystemExit: 2

/home/ivy/anaconda3/envs/nzk/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [185]:
lr = 0.001
Ks = 2
weight_decay_rate = 0.0005
n_his = 12
opt = 'adam'
fname = './figure/hyper/STGCN Learning Curves-Embeds_ + ' + "lr: " + str(lr) + " Ks: " + str(Ks) + " weight_decay_rate: " + str(weight_decay_rate) + " n_his: " + str(n_his) + "opt: " + opt 

history = [0.6949383338014007, 0.5823424432549749]
with open (fname+'.txt' , "w") as f:
    f.write(str(history))